In [1]:
# Se importan las librerias necesarias
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import date

In [2]:
# Se carga la pagina para realizar el scraping

url = 'https://cuspide.com/100-mas-vendidos/'
r = requests.get(url)

if r.status_code == 200:
    # El código 200 indica que la solicitud fue exitosa
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
else:
    print("Error en la solicitud web. Código de estado:", r.status_code)

In [3]:
# Despues de localizar las etiquetas se procede a extraer los enlaces de cada libro y el nombre

links = soup.find_all('a')
enlaces = []
nombres = []
for link in links:
        href = link.get('href')
        if href and '/producto/' in href:
            enlace = href
            nombre = link.text.strip()
            enlaces.append(enlace)
            nombres.append(nombre)

# Este codigo se hizo porque los enlaces salen repetidos
enlaces_sd = []
for j in enlaces:
    if j not in enlaces_sd:
        enlaces_sd.append(j)

In [4]:
# Se realiza una limpieza a la lista de nombres
nombres = [elemento.replace(".", "").strip() for elemento in nombres]
nombres = [elemento for elemento in nombres if elemento]


# Se hace el scraping para obtener los precios en pesos argentinos
bdi_elements = soup.find_all('bdi')
precio_pesos = []
# Recorre la lista de elementos y extrae su contenido de texto
for bdi_element in bdi_elements:
    precio = bdi_element.text.strip()
    precio_pesos.append(precio)

# Se realiza una limpieza al precio
precio_pesos = [elemento.replace('$', '').replace('.', '').replace(',', '.').strip() for elemento in precio_pesos]
precio_pesos_array = np.array(precio_pesos, dtype=float)

# Se ellimina el primer valor
precio_pesos_array = np.delete(precio_pesos_array, 0)

In [5]:
# Se hace el scraping del precio de cada libro en dolares $US

precio_dolares = []  # Aquí almacenaremos los precios en dólares

for url_libro in enlaces_sd:
    try:
        precio_response = requests.get(url_libro)
        if precio_response.status_code == 200:
            html_precio = precio_response.text
            precio_bus = BeautifulSoup(html_precio, features='html.parser')
            precio_libro_usd = precio_bus.find('span', style='font-size: 1.3em')

            if precio_libro_usd:
                precio_dolares.append(precio_libro_usd.text.strip().replace(',','.'))
            else:
                print(f'No se encontró el precio en la página: {url_libro}')
        else:
            print(f'Error en la solicitud de la página: {url_libro}')
    except Exception as e:
        print(f'Error al procesar el enlace {url_libro}: {str(e)}')



In [6]:
# Hacemos la carga de la pagina donde se extraera el precio del dolar Blue

url = 'https://www.cronista.com/MercadosOnline/dolar.html'
r = requests.get(url)

if r.status_code == 200:
    # El código 200 indica que la solicitud fue exitosa
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    # Resto del código para extraer los nombres de los libros
else:
    print("Error en la solicitud web. Código de estado:", r.status_code)

In [7]:
# Se hace el scraping en la pagina

div_sell_values = soup.find_all('div', class_='sell-value')

# Inicializa una lista para almacenar los precios
precios = []

# Recorre los dos primeros elementos <div> (precios)
for div in div_sell_values[:2]:
    # Busca el elemento <span> dentro de <div> para obtener el precio
    span = div.find('span', class_='currency')
    precio = div.text.strip()

    # Agrega el precio a la lista de precios
    precios.append(precio)

# Ahora, precios[1] contienen el precio del dolar Blue
precios = [e.replace('$', '').replace(',', '.').strip() for e in precios]
precios_array = np.array(precios, dtype=float)


dolar_blue = precios_array[1]

precio_dolar_blue = np.round(precio_pesos_array / dolar_blue, decimals=2)

print('Dolar Blue:', dolar_blue)

# Se pone la fecha del scraping 
fecha = date.today()



Dolar Blue: 925.0


In [18]:
datos = {'titulo': nombres, 'url': enlaces_sd, 'precio': precio_pesos_array, 'precio_usd': precio_dolares, 'precio_usd_blue': precio_dolar_blue, 'fecha': fecha}
df = pd.DataFrame(datos)
#df.head()
df.to_csv('cuspide.csv', index=False, encoding='utf-8')


##  CARGA DE LA BASE DE DATOS

In [19]:
import pymysql
import csv

In [25]:
# Leer los datos de conexión desde el archivo de configuración
with open("config.txt", "r") as config_file:
    config = {}
    for line in config_file:
        key, value = line.strip().split("=")
        config[key] = value

# Establecer la conexión utilizando los datos de configuración
conexion = pymysql.connect(host=config["Host"], user=config["User"], passwd=config["Password"], db=config["Database"])
cursor = conexion.cursor()

In [32]:
# Crea una base de datos
cursor.execute("DROP TABLE IF EXISTS cuspide")
cursor.execute("CREATE DATABASE IF NOT EXISTS cuspide")

# Selecciona la base de datos recién creada
cursor.execute("USE cuspide")

# Se crea la tabla
cuspide =   """CREATE TABLE IF NOT EXISTS top_100 (
                    IdLibro           INT AUTO_INCREMENT,
                    titulo             VARCHAR(150),
                    url                VARCHAR(180),
                    precio             DECIMAL(10,2),
                    precio_usd         DECIMAL(8,2),
                    precio_usd_blue    DECIMAL(8,2),
                    fecha              DATE,
                    PRIMARY KEY (IdLibro)
                    );
                    """


# Ejecuta la consulta para crear la tabla
cursor.execute(cuspide)

# Confirma los cambios en la base de datos
conexion.commit()

In [23]:
# Abre el archivo CSV y lee los datos
with open("cuspide.csv", "r", newline="") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    for row in csv_reader:
        titulo = row["titulo"]
        url = row["url"]
        precio = row["precio"]
        precio_usd = row["precio_usd"]
        precio_usd_blue = row["precio_usd_blue"]
        fecha = row["fecha"]
        
        # Insertar los datos en la tabla
        cursor.execute("INSERT INTO top_100 (titulo, url, precio, precio_usd, precio_usd_blue, fecha) VALUES (%s, %s, %s, %s, %s, %s)",
                       (titulo, url, precio, precio_usd, precio_usd_blue, fecha))

# Confirma los cambios en la base de datos
conexion.commit()

# Cierra la conexión
conexion.close()
